**Однонаправленная LSTM с посимвольной токенизацией:**

In [ ]:
import numpy as np
import os

import tensorflow as tf
from tensorflow.data import Dataset, AUTOTUNE
from tensorflow import keras
import keras.layers as l

from keras import models, callbacks, utils, losses


In [ ]:
from typing import List
from bs4 import BeautifulSoup
import requests
import os
def request_url(url: str) -> BeautifulSoup:
    request = requests.get(url)
    soup = BeautifulSoup(request.content, 'html.parser')
    return soup


def get_url_data(url: str) -> List[str]:
    soup = request_url(url)
    scrapped_text = []

    h1 = soup.h1.text.strip()
    p = soup.find_all('p')

    scrapped_text.append(h1)
    scrapped_text.extend([p_i.text.strip() for p_i in p])

    return scrapped_text

def get_data(url: str) -> str:
    soup = request_url(url)
    text = []

    text.extend([
        soup.h1.text.strip() + '.',
        soup.h2.text.strip() + '.',
        soup.article.p.text.strip()
    ])

    url_chapters = [link.get('href') for link in soup.find_all('a', class_='link')]

    for url in url_chapters:
        scrapped_text = get_url_data(url)
        text.extend(scrapped_text)

    text = ' '.join(text).lower()

    return text

In [ ]:
def load_data(url: str, file_name: str, path_dir: str = 'data/') -> str:
    if os.path.isdir(path_dir) == False:
        os.mkdir(path_dir)
        print(f'Created {path_dir} directory')

    path_file = f'{path_dir}{file_name}'

    try:
        with open(path_file, 'r', encoding='utf-8') as file:
            text = file.read()

        print('Uploaded from', path_file)

    except:
        text = get_data(url)

        with open(path_file, 'w', encoding='utf-8') as file:
            file.write(text)

        print('Saved to', path_file)

    return text

In [ ]:
text = load_data('https://hpmor.ru/', 'hpmor.txt')

Created data/ directory
Saved to data/hpmor.txt


**Токенизация:**

In [ ]:
dictionary = np.array(sorted(set(text)))
dictionary

array(['\n', ' ', '!', '#', '$', '%', '&', '(', ')', '*', '+', ',', '-',
       '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':',
       ';', '=', '?', '[', '\\', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f',
       'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's',
       't', 'u', 'v', 'w', 'x', 'y', 'z', '«', '»', '×', 'é', 'ð', 'ó',
       'þ', '́', 'а', 'б', 'в', 'г', 'д', 'е', 'ж', 'з', 'и', 'й', 'к',
       'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч',
       'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я', 'ё', 'ѝ', '–', '—', '’',
       '“', '”', '„', '…', '№', '∀', '∄', '−'], dtype='<U1')

In [ ]:
tokenizer_word_to_index_symbol = {char: i for i, char in enumerate(dictionary)}
tokenizer_index_to_word_symbol = {i: char for i, char in enumerate(dictionary)}

**Формирование датасета:**

In [ ]:
vectorized_text = np.array([tokenizer_word_to_index_symbol[char] for char in text])

In [ ]:
vectorized_text = Dataset.from_tensor_slices(vectorized_text)

In [ ]:
VOCAB_LEN = len(tokenizer_word_to_index_symbol.items())
BATCH_SIZE = 100

In [ ]:
sequences = vectorized_text.batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
BATCHES_PER_EPOCH = len(sequences) // BATCH_SIZE

In [ ]:
from typing import Tuple, List
def get_features_target(seq: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
    features = seq[:-1]
    target = seq[1:]
    return features, target

In [ ]:
df = sequences.map(get_features_target).batch(BATCH_SIZE, drop_remainder=True).repeat()
df = df.prefetch(AUTOTUNE)

**Построение и обучение модели:**

In [ ]:
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(
    max_tokens=len(dictionary),
    input_shape=(1,))

In [ ]:
import keras.layers as l
model = keras.Sequential([
    l.Embedding(len(dictionary), BATCH_SIZE, batch_input_shape=[BATCH_SIZE, None]),
    l.LSTM(64, return_sequences=True, stateful=True),
    l.Dense(len(dictionary))
])

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (100, None, 100)          11200     
                                                                 
 lstm_1 (LSTM)               (100, None, 64)           42240     
                                                                 
 dense_1 (Dense)             (100, None, 112)          7280      
                                                                 
Total params: 60720 (237.19 KB)
Trainable params: 60720 (237.19 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.compile(
    optimizer='adam',
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [ ]:
preds = model.fit(
    df,
    epochs=100,
    steps_per_epoch=BATCHES_PER_EPOCH
)

Epoch 1/100
360/360 [==============================] - 17s 38ms/step - loss: 3.0227 - accuracy: 0.2031
Epoch 2/100
360/360 [==============================] - 10s 28ms/step - loss: 2.5494 - accuracy: 0.2631
Epoch 3/100
360/360 [==============================] - 10s 27ms/step - loss: 2.4260 - accuracy: 0.2935
Epoch 4/100
360/360 [==============================] - 8s 22ms/step - loss: 2.3170 - accuracy: 0.3280
Epoch 5/100
360/360 [==============================] - 10s 27ms/step - loss: 2.2249 - accuracy: 0.3534
Epoch 6/100
360/360 [==============================] - 9s 25ms/step - loss: 2.1474 - accuracy: 0.3742
Epoch 7/100
360/360 [==============================] - 8s 23ms/step - loss: 2.0841 - accuracy: 0.3911
Epoch 8/100
360/360 [==============================] - 9s 26ms/step - loss: 2.0321 - accuracy: 0.4055
Epoch 9/100
360/360 [==============================] - 10s 28ms/step - loss: 1.9872 - accuracy: 0.4187
Epoch 10/100
360/360 [==============================] - 8s 22ms/step - loss: 